In [15]:
using Plots
include("datasetloader.jl")
include("ngrams.jl")

load_vocab (generic function with 1 method)

In [2]:
function norm_table!(t)
    D = last(t)
    vs = sum(values(D))
    for (k, v) in D
        D[k] /= vs
        D[k] = log(D[k])
    end
end

norm_table! (generic function with 1 method)

In [3]:
GS = Dict()
for lang in LANGS
    GS[lang] = load_ngrams(joinpath("ngrams0", lang * ".txt"))
end
G_sum = sum(values(GS))
all_codes = collect(keys(G_sum));
G_avg = (1/50)*G_sum

Dict{Vector{UInt8}, Float64} with 910697 entries:
  [0x69, 0x66, 0x65, 0x6d]       => 4.31432e-7
  [0x90, 0xd7, 0xa1, 0xd7, 0x9b] => 3.61445e-7
  [0x74, 0x6e, 0x69, 0x6b, 0x20] => 2.87636e-6
  [0x6c, 0x6b, 0x61, 0xe1]       => 4.58639e-7
  [0x82, 0x92, 0xe3, 0x83, 0x95] => 9.56918e-7
  [0x67, 0x68, 0x61, 0x73]       => 4.7093e-6
  [0x81, 0x8c, 0xe5, 0xa6]       => 2.7065e-7
  [0x65, 0x6d, 0x75, 0x65, 0x20] => 7.55738e-8
  [0x83, 0xd9, 0x85, 0xd8, 0xa7] => 1.35521e-5
  [0x73, 0x69, 0x65, 0x72, 0x6f] => 4.52921e-6
  [0x88, 0x9b]                   => 6.28565e-6
  [0xb1, 0x6d, 0x20, 0x67, 0xc3] => 8.96549e-7
  [0xa3, 0x20, 0x79, 0x65, 0x66] => 7.34826e-7
  [0x67, 0x72, 0x75, 0x64, 0x20] => 3.49421e-6
  [0xa7, 0xd7, 0x94]             => 7.78452e-5
  [0x83, 0x6c, 0xc4, 0x83, 0x74] => 7.48529e-6
  [0x75, 0x73, 0x62, 0x61, 0x6e] => 6.16668e-6
  [0x74, 0x65, 0x6e, 0x62, 0x65] => 1.76262e-6
  [0xaf, 0xe7, 0x9a]             => 2.13368e-5
  ⋮                              => ⋮

In [4]:
sort([l=>sum(values(D)) for (l, D) in GS], by=last, rev=true)

50-element Vector{Pair{String, Float64}}:
 "rus" => 35.58628331397368
 "eng" => 34.92227553146798
 "deu" => 34.27479354056778
 "ita" => 34.02995117687345
 "tur" => 33.97563673228078
 "fra" => 33.68225241840984
 "heb" => 33.557930400893056
 "spa" => 33.394733791885514
 "por" => 33.38888763544054
 "jpn" => 33.379976385875686
       ⋮
 "tat" => 29.50726455626444
 "nor" => 29.500759446814232
 "nds" => 29.476489870775552
 "vie" => 29.43076878060164
 "kor" => 29.204329962552535
 "cat" => 29.072319269071862
 "isl" => 28.894979639764003
 "slk" => 28.702952882802666
 "ido" => 28.034869375267558

In [5]:
norm_table!.(collect(GS));
norm_table!(1=>G_avg);
DEFAULT_LOGP = (minimum.(values.(last.(collect(GS))))|>minimum)


-17.203694184405613

In [6]:
code_score(code) = only(diff(extrema([get(D, code, DEFAULT_LOGP) for D in values(GS)])|>collect)) + G_avg[code]

code_score (generic function with 1 method)

In [7]:
code_std = [code => code_score(code) for code in all_codes]

910697-element Vector{Pair{Vector{UInt8}, Float64}}:
       [0x69, 0x66, 0x65, 0x6d] => -15.108885839292851
 [0x90, 0xd7, 0xa1, 0xd7, 0x9b] => -15.506856881576052
 [0x74, 0x6e, 0x69, 0x6b, 0x20] => -11.555829800658078
       [0x6c, 0x6b, 0x61, 0xe1] => -14.986576992986189
 [0x82, 0x92, 0xe3, 0x83, 0x95] => -13.554325575906388
       [0x67, 0x68, 0x61, 0x73] => -10.255846273471466
       [0x81, 0x8c, 0xe5, 0xa6] => -16.08011029783409
 [0x65, 0x6d, 0x75, 0x65, 0x20] => -18.640556407123263
 [0x83, 0xd9, 0x85, 0xd8, 0xa7] => -8.189557579968614
 [0x73, 0x69, 0x65, 0x72, 0x6f] => -11.321330333030584
                                ⋮
 [0x75, 0x69, 0x20, 0x6b, 0x75] => -12.179512001326154
 [0x72, 0x20, 0x67, 0x65, 0xc3] => -11.73866652707466
 [0x66, 0x65, 0x63, 0xc3, 0xa7] => -13.716671493168084
       [0x91, 0x97, 0xe6, 0x88] => -13.193097974678645
 [0x7a, 0x61, 0x6e, 0x20, 0x69] => -7.236198481585225
 [0xa6, 0x97, 0xe0, 0xa6, 0xb8] => -11.386570883930121
 [0x86, 0xd9, 0x81, 0xdb, 0x8c] => -1

In [8]:
sorted_codes = sort(code_std, by=last, rev=true)

910697-element Vector{Pair{Vector{UInt8}, Float64}}:
                         [0xd0] => 9.789988328246345
                         [0xd7] => 9.189355800444918
                         [0xe0] => 8.540854352603082
                         [0xa4] => 8.53352635405076
                         [0xd1] => 8.398282134091275
                         [0xd8] => 8.18872858983763
                   [0xe0, 0xa4] => 8.132026818849134
                   [0x20, 0xe0] => 7.970080736465004
                         [0xa6] => 7.748458490711555
                         [0xce] => 7.711652084296028
                                ⋮
 [0x77, 0x64, 0x65, 0x72, 0x65] => -21.005582567753024
 [0x74, 0x79, 0x20, 0x75, 0x62] => -21.005582567753024
 [0x70, 0x79, 0x20, 0x73, 0x74] => -21.005582567753024
 [0x74, 0x6f, 0x6f, 0x70, 0x65] => -21.005582567753024
 [0x6d, 0x20, 0x62, 0x75, 0x66] => -21.005582567753024
 [0x65, 0x20, 0x63, 0x64, 0x73] => -21.005582567753024
 [0x76, 0x6f, 0x72, 0x61, 0x63] => -21.005582567753024


In [9]:
vocab = first.(sorted_codes[1:20000]);

In [10]:
dump_vocab(vocab, "vocab.txt")

In [11]:
vocab = Set(vocab);

In [12]:
GS = Dict()
for lang in LANGS
    GS[lang] = load_ngrams(joinpath("ngrams0", lang * ".txt"))
end

In [13]:
GSF = Dict()
for (lang, D) in GS
    DF = Dict(k=>v for (k,v) in D if k in vocab)
    GSF[lang] = DF
end

In [16]:
mkpath("ngrams")
for (lang, D) in GSF
    dump_ngrams( sort(D|>collect, by=last, rev=true), joinpath("ngrams", lang * ".txt"))
end

In [19]:
sum(length.(values(GSF)))/50

3923.02